# Расчёт данных по тематикам (объединению проектов)

## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()


## Формируем задания

### Общие параметры для заданий

In [ ]:
# Задаем параметры
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop-pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2021-03-01'
date_to = '2021-03-31'

# Задаем список статистик для расчета
statistics=['Reach', 'ReachPer', 'ADR', 'ADRPer', 'UnwReach']

# Важно!
# Если в демографической переменной используется возраст, то переменной ages должно быть присвоено значение None
# Если в демографической переменной используется численность населения города, то переменной population должно быть присвоено значение None

# задаем Население           
population = rcats.get_population('Russia0+')

# задаем Возврастные группы
ages = rcats.get_age_groups('12+')

# Задаем демо фильтр
demo_filter = None

# Создадим словарь для нужных нам типов пользования Интернетом:
ut = {'Desktop&Mobile':[1,2,3,4], 'Desktop':[1], 'Mobile':[2,3,4]}

# Проверяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

## Создаем словарь с нужными проектами

Для этого подготовим тематики и проекты, входящие в них, в excel-файле. Для примера смотрите файл: __Состав тематик.xlsx__ в папке с ноутбуками

__ВАЖНО!__ Словарь нужно сохранить в папку с ноутбуками
 
__При изменении названия файла/листа/колонок, следует внести изменения в код ниже - указать правильное название листа и колонок:__

In [ ]:
sostav = pd.read_excel('Состав тематик.xlsx', sheet_name='тематики')[['Тематика', 'Для словаря']]

topics = {}

for _, t in sostav.iterrows():
    if t['Тематика'] not in topics:
        topics[t['Тематика']] = f"{t['Для словаря']} OR "
    else:
        topics[t['Тематика']] += f"{t['Для словаря']} OR "
for key, value in topics.items():
    topics[key] = value[:-4]
print('Словарь готов')

## Расчёт тематик Total

In [ ]:
%%time

#Посчитаем данные в циклах
tasks = []
print('Отправка задач ')

for utype, usetypes in ut.items():
    for topic, filter_obj in topics.items():
        project_name = f"{topic}"
        media_filter = f"{filter_obj}"
        structure = {
            "date": "month",
            "usetype": False,
        }

        # Формируем из заданных параметров заданние для Responsum в формате JSON для каждого из этапов цикла
        task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['usetype'] = f"{utype}"
        tsk['task'] = rtask.send_audience_task(task_json) 
        tasks.append(tsk)
        time.sleep(2)
        print('.', end = '')
print('')

# Ждем выполнения
tsks = rtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

#Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df = rtask.result2table(rtask.get_result(tsk), project_name = t['project_name'])
    df['usetype'] = t['usetype']
    results.append(df)
    print('.', end = '')
df_result = pd.concat(results)
df_result

#### Сохраняем полученные результаты в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('Тематики'))
df_result.to_excel(writer, 'Тематики', index=False)
writer.save()

## Расчёт тематик с разбивкой на проекты

In [ ]:
%%time

#Посчитаем данные в циклах]
tasks_sep = []
print('Отправка задач ')

for utype, num in ut.items():
    for topic, filter_obj in topics.items():
        usetypes = num
        project_name = f"{topic}"
        media_filter = f"{filter_obj}"
        structure = {
            "date": "month",
            "media": ["site"],
            "usetype": False,
        }

        # Формируем из заданных параметров заданние для Responsum в формате JSON для каждого из этапов цикла
        task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

        # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
        tsk = {}
        tsk['project_name'] = project_name
        tsk['usetype'] = f"{utype}"
        tsk['task'] = rtask.send_audience_task(task_json)
        tasks_sep.append(tsk)
        time.sleep(2)
        print('.', end = '')

# Ждем выполнения
tsks = rtask.wait_task(tasks_sep)
print('Расчет завершен, получаем результат')

#Получаем результат
results_sep = []
print('Собираем таблицу')
for t in tasks_sep:
    tsk = t['task'] 
    df_result_sep = rtask.result2table(rtask.get_result(tsk), project_name = t['project_name'])
    df_result_sep['usetype'] = t['usetype']
    results_sep.append(df_result_sep)
    print('.', end = '')
df_sep = pd.concat(results_sep)
df_sep

#### Сохраняем полученные результаты в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('Тематики с разбивкой на проекты'))
df_sep.to_excel(writer, 'Тематики', index=False)
writer.save()